# SMU Logo bounding box detection
Design an algorithm to identify all images of SMU from a set of random images. A bounding box should be drawn around the SMU logo whenever it appears in an image.

When an SMU logo occurs in an image, a bounding box should be drawn around it. Logo identification will be evaluated by its F1 score.

### Setup
Install dependencies
```bash
pip install -r requirements.txt

```

In [ ]:
%pip install -r requirements.txt

Import packages

In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow
from PIL import Image
from dotenv import load_dotenv
import os

Load environment variables using dotenv
- `API_KEY` - API Key for Roboflow dataset
- `DATA_VERSION` - Version of dataset used
- `PATH_TO_DATA` - Absolute file path to the "SMU-Logo-Detection-1" folder. See .env.example for example

In [ ]:
# Project settings
load_dotenv()
api_key = os.getenv('API_KEY')
data_version = os.getenv('DATA_VERSION')
path_to_data = f'{os.getenv("PATH_TO_PROJECT")}/SMU-Logo-Detection-{data_version}'

Load dataset into workspace

In [ ]:
rf = Roboflow(api_key=api_key)
project = rf.workspace("cs424").project("smu-logo-detection")
dataset = project.version(data_version).download("yolov8")


Update test, train and val values in data.yaml
This fixes a file not found bug

In [ ]:
!python update_path.py

### [Training](https://docs.ultralytics.com/modes/train/#train-settings)
Use a trained model, saved in `/saved_models`, or train a model.
A pre-trained [model](https://github.com/ultralytics/ultralytics?tab=readme-ov-file) `yolov8n.pt` from Ultralytics should be used. Tune hyper-params such as learning rate and epochs.

If there is file not found error, check the `FILE_TO_PATH` env variable, and make sure that you've updated the `data.yml` to the absolute file path.

In [ ]:
# # Train model (Laptop)
# model = YOLO('yolov8n.pt') # pre-trained yolov8 nano
# results = model.train(data=f'{path_to_data}/data.yaml', epochs=10)

# # Train model (SMU GPU)
# model = YOLO('yolov8n.pt')
# results = model.train(data=f'{path_to_data}/data.yaml', epochs=10, batch=-1, device=0, workers=4)

In [ ]:
# Use saved model
model = YOLO('./saved_models/best_violet.pt')

After training, the model can be found in `/runs`. Save the `best.pt` model in the `/saved_models` folder.

### [Validation](https://docs.ultralytics.com/modes/val/#usage-examples)
Val mode is used for validating a YOLOv8 model after it has been trained. In this mode, the model is evaluated on a validation set to measure its accuracy and generalization performance. This mode can be used to tune the hyperparameters of the model to improve its performance.

In [ ]:
model.val(data=f'{path_to_data}/data.yaml', conf=0.25, iou=0.6)

### [Test](https://docs.ultralytics.com/modes/predict/#why-use-ultralytics-yolo-for-inference)
Predict bounding boxes on test set

In [ ]:
# Predict
prediction = model.predict(source=f"{path_to_data}/test/images", conf=0.25, iou=0.6)

Save images to `/predictions` folder if bounding box exists.

In [ ]:
im_rgb = prediction[62].plot(line_width=1)
im_rgb = Image.fromarray(im_rgb[..., ::-1])  # RGB-order PIL image
im_rgb.save(fp=f'{path_to_data}/../predictions/result{0}.jpg')


In [ ]:
images = {}
for i in range(0, len(prediction)):
    if prediction[i].boxes.shape[0] >= 1:
        im_rgb = prediction[i].plot(line_width=1)
        im_rgb = Image.fromarray(im_rgb[..., ::-1])  # RGB-order PIL image
        images[i] = im_rgb
        im_rgb.save(fp=f'{path_to_data}/../predictions/result{i}.jpg')
images